In [1]:
# coding: utf-8
%load_ext autoreload
%autoreload 2

In [2]:
import h1st as h1
h1.init()

In [3]:
from h1st.tuner import HyperParameterTuner
from examples.RayTune.tensorflow_mlp_classifier import TensorflowMLPClassifier
from examples.RayTune.sklearn_rf_classifier import SklearnRFClassifier

In [4]:
tuner = HyperParameterTuner()

In [5]:
# parameters = [
#    
#         'name': 'lr',
#         'type': 'float',
#         'min': 0.001,
#         'max': 0.1,
#         'choice': []
#     },
#     {
#         'name': 'n_layer',
#         'type': 'int',
#         'min': 1,
#         'max': 8,
#         'choice': []
#     },
#     {
#         'name': 'units',
#         'type': 'int',
#         'min': None,
#         'max': None,
#         'choice': [4, 8, 16, 32, 64]
#     }    
# ]

parameters = [
    {
        'name': 'n_estimators',
        'type': 'int',
        'min': None,
        'max': None,
        'choice': [50, 100, 150]
    },
    {
        'name': 'max_depth',
        'type': 'int',
        'min': 3,
        'max': 8,
        'choice': []
    },
    {
        'name': 'criterion',
        'type': 'str',
        'min': None,
        'max': None,
        'choice': ['gini', 'entropy']
    },
    {
        'name': 'min_samples_split',
        'type': 'int',
        'min': 3,
        'max': 5,
        'choice': []        
    }
]


target_metrics = 'accuracy'
options = {
    'num_samples': 32,
    'stopping_criteria': {
        'training_iteration': 3,
    },
    'max_concurrent': 8 #  default should be the number of cpu
}

If it is a choice, then we need to use other model 

In [6]:
analysis = tuner.run(SklearnRFClassifier, parameters, target_metrics, options, search_algorithm="RANDOM", verbose=1)

Trial name,status,loc,criterion,max_depth,min_samples_split,n_estimators,iter,total time (s),accuracy
H1stModelTrainable_356b0_00000,TERMINATED,,entropy,5,3,50,3,6.21543,0.810101
H1stModelTrainable_356b0_00001,TERMINATED,,entropy,5,3,150,3,19.094,0.809697
H1stModelTrainable_356b0_00002,TERMINATED,,entropy,4,4,150,3,15.3599,0.808283
H1stModelTrainable_356b0_00003,TERMINATED,,entropy,3,3,50,3,3.75143,0.806061
H1stModelTrainable_356b0_00004,TERMINATED,,entropy,6,3,150,3,22.8822,0.817576
H1stModelTrainable_356b0_00005,TERMINATED,,gini,6,3,100,3,12.9318,0.813535
H1stModelTrainable_356b0_00006,TERMINATED,,entropy,6,4,100,3,15.4959,0.815253
H1stModelTrainable_356b0_00007,TERMINATED,,entropy,5,4,100,3,12.149,0.810404
H1stModelTrainable_356b0_00008,TERMINATED,,gini,5,4,150,3,16.0943,0.811717
H1stModelTrainable_356b0_00009,TERMINATED,,entropy,3,3,150,3,11.8085,0.805152


took   57.51 seconds (  0 hours 0.96 minutes)
best config:  {'n_estimators': 100, 'max_depth': 7, 'criterion': 'entropy', 'min_samples_split': 3}


In [7]:
analysis.head(20)

,model_version,accuracy,n_estimators,max_depth,criterion,min_samples_split,training_iteration
27,n_e=100|max=7|cri=entropy|min=3|accu=0.8192,0.819192,100,7,entropy,3,3
24,n_e=100|max=7|cri=gini|min=4|accu=0.8191,0.819091,100,7,gini,4,3
10,n_e=100|max=7|cri=entropy|min=4|accu=0.8190,0.818990,100,7,entropy,4,1
30,n_e=50|max=5|cri=gini|min=4|accu=0.8188,0.818788,50,5,gini,4,1
5,n_e=100|max=6|cri=gini|min=3|accu=0.8183,0.818283,100,6,gini,3,2
4,n_e=150|max=6|cri=entropy|min=3|accu=0.8182,0.818182,150,6,entropy,3,1
13,n_e=100|max=7|cri=gini|min=4|accu=0.8182,0.818182,100,7,gini,4,1
19,n_e=100|max=6|cri=gini|min=4|accu=0.8181,0.818081,100,6,gini,4,2
14,n_e=150|max=7|cri=gini|min=4|accu=0.8180,0.817980,150,7,gini,4,2
6,n_e=100|max=6|cri=entropy|min=4|accu=0.8178,0.817778,100,6,entropy,4,2
